# **Random Forest**
- Using data that was not annotated
- Training it on the unbalanced data
- Using avgpool
- 6 second window

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Standard libraries
import numpy as np
import pandas as pd
import time
import os

# For audio
from IPython.display import Audio
import librosa

# For preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

# For modeling
from sklearn.metrics import classification_report, roc_auc_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
from scipy.signal import butter, filtfilt
import random

In [3]:
pkl_path = '/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/feature-extraction/NotAnnotated/Regular/AveragePooled/split_features_6s_all_1D.pkl'

# Load the pickle file
with open(pkl_path, 'rb') as file:
    data = pickle.load(file)

In [4]:
train_data = data['train'].copy()
val_data = data['val'].copy()

In [5]:
train_data

{'melspectrogram': array([[-0.01776731, -0.989827  , -0.99071366, ..., -0.8216415 ,
         -0.85376334, -0.9824129 ],
        [-0.01776753, -0.9898328 , -0.99071616, ..., -0.8170307 ,
         -0.84770477, -0.9777302 ],
        [-0.01776784, -0.989846  , -0.99073434, ..., -0.82816565,
         -0.8608326 , -0.98463976],
        ...,
        [-0.01781281, -0.5766479 , -0.54141074, ..., -0.87102264,
         -0.89623916, -0.99998677],
        [-0.01774438, -0.57157916, -0.5401701 , ..., -0.8684321 ,
         -0.89438707, -0.99997807],
        [-0.01779194, -0.5788874 , -0.54096264, ..., -0.8674551 ,
         -0.89504766, -0.999996  ]], dtype=float32),
 'mfcc': array([[-1.        , -0.10011531, -0.14837739, ...,  0.02309681,
          0.0204311 ,  0.04844239],
        [-1.        , -0.11325053, -0.18573974, ...,  0.02437621,
          0.02035097,  0.05108224],
        [-1.        , -0.10374827, -0.18394189, ...,  0.02396246,
          0.01970905,  0.04918307],
        ...,
        [-0.9

In [6]:
train_labels = train_data['label'].copy()
temp = train_data.copy()
del temp['label']
tr_features = temp

In [7]:
val_labels = val_data['label'].copy()
temp = val_data.copy()
del temp['label']
v_features = temp

# Shuffle Data

In [8]:
def shuffle_data(input_label, input_features):
  input_len = len(input_label)
  np.random.seed(1826)
  input_indices = np.random.permutation(input_len)
  input_features = {key: np.array([input_features[key][i] for i in input_indices]) for key in input_features} # dictionary comprehension
  input_label = np.array([input_label[i] for i in input_indices])

  return input_label, input_features

In [9]:
train_y, train_features = shuffle_data(train_labels, tr_features)

In [10]:
val_y, val_features = shuffle_data(val_labels, v_features)

In [11]:
display(train_y.shape)
display(train_y[:15])

(6543,)

array([19,  2,  7,  1,  2, 19,  7,  7,  0, 18, 19, 12,  0, 15, 12],
      dtype=int32)

In [12]:
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)
  display(train_features[key][0])

'melspectrogram'

(6543, 128)

array([-0.01776706, -0.98747253, -0.982526  , -0.96910095, -0.942215  ,
       -0.9638503 , -0.9875802 , -0.9905174 , -0.9911741 , -0.99101436,
       -0.9906444 , -0.9879602 , -0.9845477 , -0.97809017, -0.9798955 ,
       -0.98365515, -0.9822163 , -0.97887224, -0.98426366, -0.98790485,
       -0.9786248 , -0.9810127 , -0.98267984, -0.98049825, -0.98332447,
       -0.9814118 , -0.981292  , -0.9789223 , -0.9792321 , -0.98115015,
       -0.98421574, -0.9866537 , -0.9838543 , -0.9709954 , -0.9539507 ,
       -0.94942206, -0.9324423 , -0.9088786 , -0.89429635, -0.87762254,
       -0.86245364, -0.83916736, -0.80800724, -0.7875475 , -0.7559936 ,
       -0.72916424, -0.7109448 , -0.6965236 , -0.68793595, -0.67658234,
       -0.6674314 , -0.65027046, -0.63032126, -0.6145561 , -0.5971278 ,
       -0.5862773 , -0.5772054 , -0.5720244 , -0.574622  , -0.57558393,
       -0.5744448 , -0.5712204 , -0.56524533, -0.5525866 , -0.5526075 ,
       -0.5596768 , -0.5844143 , -0.60527414, -0.62253195, -0.63

'mfcc'

(6543, 20)

...

(6543, 84)

array([-73.09822 , -73.197426, -73.273796, -73.76968 , -73.58266 ,
       -73.19705 , -73.832115, -74.14205 , -73.916565, -73.89668 ,
       -73.674866, -73.44959 , -73.76412 , -73.24343 , -73.15452 ,
       -73.37146 , -72.83074 , -71.93702 , -71.83897 , -72.02708 ,
       -71.25811 , -70.759285, -68.80554 , -68.4736  , -67.8511  ,
       -68.15875 , -69.84771 , -71.37478 , -72.96278 , -73.65958 ,
       -74.23347 , -74.917534, -75.76558 , -75.92188 , -75.96454 ,
       -75.89953 , -76.31836 , -76.23387 , -75.3822  , -74.438065,
       -73.990456, -73.06567 , -72.99704 , -73.46609 , -74.20816 ,
       -74.00004 , -73.73749 , -74.57999 , -74.83605 , -74.46016 ,
       -74.98341 , -75.22031 , -75.50244 , -75.54683 , -75.75173 ,
       -76.298225, -76.15754 , -73.803116, -71.55328 , -68.58164 ,
       -64.66023 , -60.468758, -56.253662, -52.7704  , -50.42955 ,
       -48.130302, -45.72306 , -43.541336, -42.784485, -42.18103 ,
       -41.757645, -40.58786 , -41.421165, -44.19097 , -46.699

'id'

(6543,)

'XC280984.mp3'

In [13]:
display(val_y.shape)
display(val_y[:15])

(2028,)

array([12,  7, 19,  3,  7, 13,  9, 11, 11, 12, 17,  2, 17,  3, 18],
      dtype=int32)

In [14]:
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)
  display(val_features[key][0])

'melspectrogram'

(2028, 128)

array([-0.01777715, -0.6823705 , -0.6700081 , -0.7143769 , -0.72569156,
       -0.7217254 , -0.70796573, -0.7020234 , -0.6957035 , -0.69550025,
       -0.68860817, -0.6916205 , -0.694039  , -0.70643896, -0.7069956 ,
       -0.7030926 , -0.6921738 , -0.69109046, -0.6968512 , -0.69173974,
       -0.6809189 , -0.68640286, -0.6812207 , -0.68139845, -0.67214686,
       -0.67690176, -0.675241  , -0.6680333 , -0.6716892 , -0.6699464 ,
       -0.66820323, -0.66705185, -0.6613608 , -0.66557056, -0.6653173 ,
       -0.6774219 , -0.6719049 , -0.6611124 , -0.66049206, -0.6563231 ,
       -0.6591102 , -0.64467436, -0.64172274, -0.638755  , -0.6407541 ,
       -0.6419572 , -0.6475931 , -0.6344285 , -0.6343891 , -0.64334345,
       -0.64528894, -0.63966703, -0.6269635 , -0.6387499 , -0.6530332 ,
       -0.6511723 , -0.6597135 , -0.66876924, -0.68288004, -0.6819005 ,
       -0.68560714, -0.68490344, -0.6785722 , -0.67853594, -0.6710221 ,
       -0.6644165 , -0.64187586, -0.621178  , -0.6261929 , -0.64

'mfcc'

(2028, 20)

...

(2028, 84)

array([-37.927734, -37.442028, -40.543606, -39.191135, -38.423588,
       -39.843575, -41.259094, -38.99902 , -39.11001 , -40.62095 ,
       -41.056614, -40.55615 , -39.62568 , -39.707634, -35.88653 ,
       -35.98243 , -40.28614 , -41.74624 , -40.879337, -41.84437 ,
       -43.657143, -44.65177 , -44.115776, -44.275055, -43.981037,
       -44.237938, -43.687973, -43.760662, -43.3976  , -42.919567,
       -42.142727, -42.42853 , -42.377666, -42.395542, -41.28829 ,
       -41.574146, -42.4205  , -41.430473, -41.15706 , -41.719154,
       -41.485535, -42.346863, -42.57032 , -43.071228, -42.28202 ,
       -41.79384 , -41.57667 , -41.99697 , -41.401665, -40.87029 ,
       -40.962864, -40.4783  , -40.42449 , -40.43128 , -40.65647 ,
       -40.47194 , -40.122643, -40.470573, -40.36758 , -40.06509 ,
       -39.04172 , -38.513466, -38.341705, -38.06644 , -38.18418 ,
       -38.46058 , -38.233692, -38.828243, -40.33467 , -41.545807,
       -41.532658, -40.47445 , -38.36278 , -37.622795, -38.441

'id'

(2028,)

'XC133548.mp3'

## **Random Forest Model**

Saving evaluation results

In [15]:
def evaluate_model(model, validation_features, val_y):
    # Predict class labels for validation set
    val_yhat_result = model.predict(validation_features)

    # Print classification report
    print('Validation classification Report \n')
    print(classification_report(val_y, val_yhat_result))

    # Get probabilities for the validation set (for AUC calculation)
    val_y_proba = model.predict_proba(validation_features)

    # Calculate AUC for multiclass classification using 'ovr' and 'weighted' average
    auc_score = roc_auc_score(val_y, val_y_proba, multi_class='ovr', average='weighted')
    print(f'AUC Score: {auc_score}')

    # Calculate F1-score with 'weighted' average for imbalanced dataset
    f1 = f1_score(val_y, val_yhat_result, average='weighted')
    print(f'F1 Score (Weighted): {f1}')

    # Store the scores in the dictionary
    val_score = {'f1': f1, 'auc': auc_score}

    # Return the scores dictionary
    return val_score

In [16]:
train_results = {}
val_results = {}

val_scores = {}
k=15

### **With all the features**

In [17]:
training_features = np.concatenate((train_features['mfcc'], train_features['chroma'], train_features['cqt'], train_features['melspectrogram']), axis=1)

training_features.shape

(6543, 244)

In [18]:
validation_features = np.concatenate((val_features['mfcc'], val_features['chroma'], val_features['cqt'], val_features['melspectrogram']), axis=1)

validation_features.shape

(2028, 244)

Fit the model with training data

In [19]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [20]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['all_features'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['all_features'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2682445759368836


In [21]:
val_scores['all_features'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.19      0.28      0.23       126
           1       0.33      0.53      0.41       205
           2       0.18      0.43      0.26        96
           3       0.40      0.44      0.42       152
           4       0.09      0.12      0.10        33
           5       0.00      0.00      0.00        48
           6       0.00      0.00      0.00        38
           7       0.65      0.29      0.40       107
           8       0.09      0.38      0.15        24
           9       0.15      0.22      0.17        32
          10       0.26      0.17      0.21        69
          11       0.20      0.01      0.03       272
          12       0.26      0.27      0.26       203
          13       0.09      0.03      0.04        76
          14       0.02      0.05      0.03        20
          15       0.23      0.37      0.29       119
          16       0.63      0.57      0.60   

### **With MFCCs only**

In [22]:
training_features = train_features['mfcc']

training_features.shape

(6543, 20)

In [23]:
validation_features = val_features['mfcc']

validation_features.shape

(2028, 20)

In [24]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [25]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.29980276134122286


In [26]:
val_scores['mfcc'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.22      0.55      0.31       126
           1       0.36      0.42      0.39       205
           2       0.34      0.49      0.40        96
           3       0.62      0.74      0.68       152
           4       0.00      0.00      0.00        33
           5       0.00      0.00      0.00        48
           6       0.00      0.00      0.00        38
           7       0.33      0.48      0.39       107
           8       0.15      0.50      0.23        24
           9       0.25      0.25      0.25        32
          10       0.28      0.30      0.29        69
          11       0.71      0.14      0.24       272
          12       0.27      0.17      0.21       203
          13       0.25      0.12      0.16        76
          14       0.03      0.15      0.05        20
          15       0.16      0.25      0.20       119
          16       0.46      0.33      0.39   

### **With Mel-Spectrogram Only**

In [27]:
training_features = train_features['melspectrogram']

training_features.shape

(6543, 128)

In [28]:
validation_features = val_features['melspectrogram']

validation_features.shape

(2028, 128)

In [29]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [30]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2564102564102564


In [31]:
val_scores['melspectrogram'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.22      0.33      0.26       126
           1       0.33      0.46      0.39       205
           2       0.31      0.55      0.39        96
           3       0.43      0.59      0.50       152
           4       0.09      0.15      0.11        33
           5       0.12      0.06      0.08        48
           6       0.00      0.00      0.00        38
           7       0.48      0.22      0.31       107
           8       0.11      0.46      0.17        24
           9       0.23      0.22      0.23        32
          10       0.23      0.19      0.21        69
          11       0.06      0.01      0.01       272
          12       0.19      0.16      0.17       203
          13       0.07      0.04      0.05        76
          14       0.00      0.00      0.00        20
          15       0.20      0.39      0.26       119
          16       0.44      0.58      0.50   

## **With Chroma Only**

In [32]:
training_features = train_features['chroma']

training_features.shape

(6543, 12)

In [33]:
validation_features = val_features['chroma']

validation_features.shape

(2028, 12)

In [34]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [35]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['chroma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['chroma'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.23126232741617356


In [36]:
val_scores['chroma'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.15      0.34      0.20       126
           1       0.31      0.48      0.38       205
           2       0.07      0.23      0.10        96
           3       0.61      0.69      0.65       152
           4       0.50      0.06      0.11        33
           5       0.60      0.06      0.11        48
           6       0.00      0.00      0.00        38
           7       0.15      0.20      0.17       107
           8       0.00      0.00      0.00        24
           9       0.36      0.16      0.22        32
          10       0.00      0.00      0.00        69
          11       0.15      0.01      0.01       272
          12       0.40      0.09      0.15       203
          13       0.20      0.05      0.08        76
          14       0.01      0.05      0.02        20
          15       0.29      0.39      0.33       119
          16       0.28      0.22      0.25   

## **With CQT Only**

In [37]:
training_features = train_features['cqt']

training_features.shape

(6543, 84)

In [38]:
validation_features = val_features['cqt']

validation_features.shape

(2028, 84)

In [39]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [40]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.16913214990138067


In [41]:
val_scores['cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.10      0.16      0.12       126
           1       0.46      0.47      0.47       205
           2       0.06      0.14      0.08        96
           3       0.22      0.33      0.26       152
           4       0.00      0.00      0.00        33
           5       0.00      0.00      0.00        48
           6       0.00      0.00      0.00        38
           7       0.25      0.26      0.26       107
           8       0.23      0.38      0.29        24
           9       0.02      0.03      0.02        32
          10       0.06      0.03      0.04        69
          11       0.07      0.00      0.01       272
          12       0.17      0.19      0.18       203
          13       0.18      0.25      0.21        76
          14       0.00      0.00      0.00        20
          15       0.15      0.18      0.16       119
          16       0.22      0.15      0.18   

## **With MFCCs and Mel-Spectrogram**

In [42]:
training_features = np.concatenate((train_features['mfcc'], train_features['melspectrogram']), axis=1)

training_features.shape

(6543, 148)

In [43]:
validation_features = np.concatenate((val_features['mfcc'], val_features['melspectrogram']), axis=1)

validation_features.shape

(2028, 148)

In [44]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [45]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_melspectrogram'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_melspectrogram'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.24753451676528598


In [46]:
val_scores['mfcc_melspectrogram'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.27      0.44      0.33       126
           1       0.33      0.47      0.39       205
           2       0.31      0.59      0.41        96
           3       0.44      0.61      0.51       152
           4       0.10      0.12      0.11        33
           5       0.03      0.02      0.02        48
           6       0.00      0.00      0.00        38
           7       0.38      0.22      0.28       107
           8       0.11      0.46      0.18        24
           9       0.21      0.22      0.22        32
          10       0.18      0.17      0.18        69
          11       0.17      0.01      0.02       272
          12       0.18      0.18      0.18       203
          13       0.07      0.05      0.06        76
          14       0.00      0.00      0.00        20
          15       0.24      0.38      0.30       119
          16       0.19      0.23      0.21   

## **With MFCCs and Chroma**

In [47]:
training_features = np.concatenate((train_features['mfcc'], train_features['chroma']), axis=1)

training_features.shape

(6543, 32)

In [48]:
validation_features = np.concatenate((val_features['mfcc'], val_features['chroma']), axis=1)

validation_features.shape

(2028, 32)

In [49]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [50]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_chroma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_chroma'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3155818540433925


In [51]:
val_scores['mfcc_chroma'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.22      0.72      0.34       126
           1       0.39      0.55      0.45       205
           2       0.26      0.27      0.26        96
           3       0.54      0.64      0.58       152
           4       0.00      0.00      0.00        33
           5       0.00      0.00      0.00        48
           6       0.00      0.00      0.00        38
           7       0.38      0.36      0.37       107
           8       0.14      0.50      0.22        24
           9       0.31      0.28      0.30        32
          10       0.27      0.29      0.28        69
          11       0.74      0.19      0.30       272
          12       0.44      0.20      0.27       203
          13       0.19      0.07      0.10        76
          14       0.00      0.00      0.00        20
          15       0.21      0.30      0.25       119
          16       0.51      0.47      0.49   

## **With MFCCs and CQT**

In [52]:
training_features = np.concatenate((train_features['mfcc'], train_features['cqt']), axis=1)

training_features.shape

(6543, 104)

In [53]:
validation_features = np.concatenate((val_features['mfcc'], val_features['cqt']), axis=1)

validation_features.shape

(2028, 104)

In [54]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [55]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.22830374753451677


In [56]:
val_scores['mfcc_cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.15      0.22      0.18       126
           1       0.42      0.47      0.45       205
           2       0.14      0.36      0.20        96
           3       0.32      0.40      0.35       152
           4       0.03      0.03      0.03        33
           5       0.00      0.00      0.00        48
           6       0.00      0.00      0.00        38
           7       0.27      0.31      0.29       107
           8       0.16      0.42      0.23        24
           9       0.13      0.22      0.17        32
          10       0.00      0.00      0.00        69
          11       0.18      0.01      0.02       272
          12       0.26      0.29      0.28       203
          13       0.23      0.17      0.20        76
          14       0.02      0.05      0.03        20
          15       0.20      0.21      0.20       119
          16       0.57      0.48      0.52   

## **With Chroma and CQT**

In [57]:
training_features = np.concatenate((train_features['chroma'], train_features['cqt']), axis=1)

training_features.shape

(6543, 96)

In [58]:
validation_features = np.concatenate((val_features['chroma'], val_features['cqt']), axis=1)

validation_features.shape

(2028, 96)

In [59]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [60]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['chroma_cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['chroma_cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.22140039447731755


In [61]:
val_scores['chroma_cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.17      0.29      0.21       126
           1       0.43      0.53      0.48       205
           2       0.07      0.16      0.09        96
           3       0.26      0.32      0.28       152
           4       0.00      0.00      0.00        33
           5       0.00      0.00      0.00        48
           6       0.00      0.00      0.00        38
           7       0.33      0.27      0.30       107
           8       0.21      0.42      0.28        24
           9       0.02      0.03      0.03        32
          10       0.00      0.00      0.00        69
          11       0.14      0.01      0.02       272
          12       0.21      0.18      0.19       203
          13       0.46      0.24      0.31        76
          14       0.00      0.00      0.00        20
          15       0.18      0.29      0.22       119
          16       0.53      0.71      0.60   

## **With Mel-Spectrogram and Chroma**

In [62]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['chroma']), axis=1)

training_features.shape

(6543, 140)

In [63]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['chroma']), axis=1)

validation_features.shape

(2028, 140)

In [64]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [65]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_chroma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_chroma'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2578895463510848


In [66]:
val_scores['melspectrogram_chroma'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.21      0.35      0.26       126
           1       0.35      0.44      0.39       205
           2       0.28      0.53      0.37        96
           3       0.44      0.57      0.50       152
           4       0.04      0.09      0.06        33
           5       0.15      0.08      0.11        48
           6       0.00      0.00      0.00        38
           7       0.48      0.22      0.31       107
           8       0.10      0.42      0.16        24
           9       0.24      0.25      0.24        32
          10       0.19      0.16      0.17        69
          11       0.12      0.01      0.02       272
          12       0.23      0.13      0.16       203
          13       0.07      0.03      0.04        76
          14       0.01      0.05      0.02        20
          15       0.19      0.39      0.26       119
          16       0.40      0.65      0.50   

## **With Mel-Spectrogram and CQT**

In [67]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['cqt']), axis=1)

training_features.shape

(6543, 212)

In [68]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['cqt']), axis=1)

validation_features.shape

(2028, 212)

In [69]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [70]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.24753451676528598


In [71]:
val_scores['melspectrogram_cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.15      0.19      0.17       126
           1       0.32      0.53      0.40       205
           2       0.23      0.49      0.31        96
           3       0.32      0.38      0.35       152
           4       0.06      0.06      0.06        33
           5       0.00      0.00      0.00        48
           6       0.00      0.00      0.00        38
           7       0.74      0.33      0.45       107
           8       0.12      0.46      0.19        24
           9       0.15      0.22      0.18        32
          10       0.39      0.17      0.24        69
          11       0.29      0.02      0.04       272
          12       0.20      0.21      0.20       203
          13       0.08      0.04      0.05        76
          14       0.02      0.05      0.03        20
          15       0.21      0.35      0.26       119
          16       0.57      0.46      0.51   

## **With Mel-Spectrogram, MFCCs, CQT**

In [72]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['cqt'], train_features['mfcc']), axis=1)

training_features.shape

(6543, 232)

In [73]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['cqt'], val_features['mfcc']), axis=1)

validation_features.shape

(2028, 232)

In [74]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [75]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_cqt_mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_cqt_mfcc'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2569033530571992


In [76]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.20      0.26      0.23       126
           1       0.31      0.50      0.39       205
           2       0.20      0.47      0.28        96
           3       0.37      0.43      0.39       152
           4       0.09      0.09      0.09        33
           5       0.00      0.00      0.00        48
           6       0.00      0.00      0.00        38
           7       0.62      0.29      0.39       107
           8       0.09      0.38      0.14        24
           9       0.18      0.22      0.20        32
          10       0.27      0.17      0.21        69
          11       0.25      0.02      0.03       272
          12       0.23      0.25      0.24       203
          13       0.06      0.03      0.04        76
          14       0.02      0.05      0.03        20
          15       0.24      0.34      0.28       119
          16       0.54      0.50      0.52   

## **With Mel-Spectrogram, MFCCs, Chroma**

In [77]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['chroma'], train_features['mfcc']), axis=1)

training_features.shape

(6543, 160)

In [78]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['chroma'], val_features['mfcc']), axis=1)

validation_features.shape

(2028, 160)

In [79]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [80]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_chroma_mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_chroma_mfcc'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2514792899408284


In [81]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.25      0.46      0.32       126
           1       0.36      0.46      0.40       205
           2       0.26      0.51      0.35        96
           3       0.47      0.59      0.52       152
           4       0.06      0.09      0.07        33
           5       0.03      0.02      0.02        48
           6       0.00      0.00      0.00        38
           7       0.45      0.22      0.30       107
           8       0.11      0.42      0.17        24
           9       0.19      0.22      0.20        32
          10       0.20      0.19      0.19        69
          11       0.11      0.01      0.02       272
          12       0.19      0.16      0.17       203
          13       0.09      0.04      0.05        76
          14       0.00      0.00      0.00        20
          15       0.21      0.35      0.26       119
          16       0.23      0.29      0.26   

# Review results from all models

In [82]:
train_results_df = pd.DataFrame(list(train_results.items()), columns=['Features', 'Train_Accuracy']).round(3)
val_results_df = pd.DataFrame(list(val_results.items()), columns=['Features', 'Val_Accuracy']).round(3)

result_df = train_results_df.merge(val_results_df, on='Features')
result_df = result_df.sort_values('Features')
result_df

,Features,Train_Accuracy,Val_Accuracy
0,all_features,1.0,0.268
3,chroma,1.0,0.231
8,chroma_cqt,1.0,0.221
4,cqt,1.0,0.169
2,melspectrogram,1.0,0.256
9,melspectrogram_chroma,1.0,0.258
12,melspectrogram_chroma_mfcc,1.0,0.251
10,melspectrogram_cqt,1.0,0.248
11,melspectrogram_cqt_mfcc,1.0,0.257
1,mfcc,1.0,0.300


In [83]:
val_scores_df = pd.DataFrame([(key, value['f1'], value['auc']) for key, value in val_scores.items()],
                             columns=['Features', 'F1_Score', 'AUC_Score']).round(3)

val_scores_df = val_scores_df.sort_values('Features')
print(val_scores_df)

                   Features  F1_Score  AUC_Score
0              all_features     0.247      0.703
3                    chroma     0.212      0.675
8                chroma_cqt     0.203      0.668
4                       cqt     0.157      0.649
2            melspectrogram     0.227      0.683
9     melspectrogram_chroma     0.229      0.698
10       melspectrogram_cqt     0.228      0.682
11  melspectrogram_cqt_mfcc     0.227      0.707
1                      mfcc     0.286      0.749
6               mfcc_chroma     0.301      0.770
7                  mfcc_cqt     0.212      0.677
5       mfcc_melspectrogram     0.217      0.699
